# Día 1 — Detectores de IA: Indicadores y Métricas (Colab)

Este cuaderno te guía para **calcular indicadores** (perplejidad, burstiness, estilometría) en textos y **evaluar detectores** (TP/FP/FN, Precisión, Recall, F1) desde un CSV.

## 0) Instalación de dependencias

In [2]:
!pip -q install "transformers>=4.44,<5" "nltk==3.9.1" "textstat==0.7.4" "tqdm>=4.66,<5" "matplotlib==3.9.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 151.2 MB/s eta 0:00:00


In [3]:
import pandas as pd, transformers, nltk, matplotlib
print("OK ✅ pandas", pd.__version__,
      "| transformers", transformers.__version__,
      "| nltk", nltk.__version__,
      "| matplotlib", matplotlib.__version__)

OK ✅ pandas 2.3.3 | transformers 4.57.1 | nltk 3.9.1 | matplotlib 3.10.7


## 1) Importaciones y utilidades

In [4]:
import math, re, statistics
from pathlib import Path
from typing import List, Dict
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

import textstat


## 2) Modelo ligero para **Perplejidad**

Usaremos un modelo causal (GPT-2 pequeño) para estimar **perplejidad**:
$\mathrm{PPL} = e^{\mathrm{cross\text{-}entropy}}$.

Valores **bajos** suelen indicar texto más "predecible" (común en IA); valores **altos** sugieren mayor entropía (frecuente en humanos).

In [5]:
MODEL_NAME = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.eval()
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

@torch.no_grad()
def compute_perplexity(text: str, stride: int = 512) -> float:
    """Calcula perplejidad por ventana (stride) para textos largos."""
    enc = tokenizer(text, return_tensors='pt')
    input_ids = enc['input_ids']
    nlls = []
    max_length = model.config.n_positions
    for i in range(0, input_ids.size(1), stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, input_ids.size(1))
        trg_len = end_loc - i
        input_ids_slice = input_ids[:, begin_loc:end_loc]
        target_ids = input_ids_slice.clone()
        target_ids[:, :-trg_len] = -100
        outputs = model(input_ids_slice, labels=target_ids)
        neg_log_likelihood = outputs.loss * trg_len
        nlls.append(neg_log_likelihood)
    ppl = torch.exp(torch.stack(nlls).sum() / end_loc).item()
    return float(ppl)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## 3) **Burstiness** (variabilidad entre oraciones)

Medimos la **variación relativa** en longitud de oraciones:
$\mathrm{Burstiness} = \frac{\sigma(\text{long.orac})}{\mu(\text{long.orac}) + 1e{-8}}$.

Valores **mayores** ⇒ más irregularidad humana.

In [6]:
def compute_burstiness(text: str) -> float:
    sentences = sent_tokenize(text)
    if len(sentences) <= 1:
        return 0.0
    lengths = [len(word_tokenize(s)) for s in sentences]
    return float(np.std(lengths, ddof=1) / (np.mean(lengths) + 1e-8))


## 4) **Estilometría** básica

Rasgos simples y rápidos:

- **TTR (Type-Token Ratio)** = vocabulario único / palabras totales
- **Longitud media de oración** (tokens por oración)
- **Longitud media de palabra**
- **Puntuación por 100 palabras**
- **% Funcionales** (aprox. pronombres, preps., conj.) vía POS de NLTK
- **Legibilidad** (Flesch Reading Ease; `textstat`)


In [7]:
FUNCTIONAL_TAGS = set(['PRP','PRP$','IN','CC','DT','TO','MD','UH'])

def stylometry_features(text: str) -> Dict[str, float]:
    words = [w for w in word_tokenize(text) if re.search(r'\w', w)]
    sents = sent_tokenize(text) or ['']
    if len(words) == 0:
        return {
            'ttr': 0.0,
            'sent_len_avg': 0.0,
            'word_len_avg': 0.0,
            'punct_per_100w': 0.0,
            'functional_pct': 0.0,
            'flesch': 0.0
        }
    ttr = len(set(w.lower() for w in words)) / len(words)
    sent_len_avg = np.mean([len(word_tokenize(s)) for s in sents])
    word_len_avg = np.mean([len(w) for w in words])
    punct_count = len([c for c in text if c in '.,;:!?—–-…()[]{}"\''])
    punct_per_100w = (punct_count / len(words)) * 100
    tags = [t for _, t in pos_tag(words)]
    functional_pct = (sum(1 for t in tags if t in FUNCTIONAL_TAGS) / len(tags)) * 100
    try:
        flesch = textstat.flesch_reading_ease(text)
    except Exception:
        flesch = 0.0
    return {
        'ttr': float(ttr),
        'sent_len_avg': float(sent_len_avg),
        'word_len_avg': float(word_len_avg),
        'punct_per_100w': float(punct_per_100w),
        'functional_pct': float(functional_pct),
        'flesch': float(flesch)
    }


## 5) Procesamiento por carpeta

Estructura esperada:

```
/content/textos/
├── human/
│   ├── human_01.txt
│   └── ...
└── ai/
    ├── ai_01.txt
    └── ...
```


In [ ]:
from pathlib import Path

INPUT_DIR = Path('/content/textos')  # cambia si lo necesitas
rows = []

def read_txt(path: Path) -> str:
    try:
        return path.read_text(encoding='utf-8', errors='ignore')
    except Exception:
        return path.read_text(encoding='latin-1', errors='ignore')

if INPUT_DIR.exists():
    for cls in ['human', 'ai']:
        for p in sorted((INPUT_DIR/cls).glob('*.txt')):
            txt = read_txt(p)
            ppl = compute_perplexity(txt)
            burst = compute_burstiness(txt)
            sty = stylometry_features(txt)
            rec = {
                'id': p.stem,
                'true_label': cls,
                'perplexity': ppl,
                'burstiness': burst,
                **sty
            }
            rows.append(rec)

df_feats = pd.DataFrame(rows)
if len(df_feats):
    display(df_feats.head())
    df_feats.to_csv('/content/indicadores_textos.csv', index=False)
    print('Guardado: /content/indicadores_textos.csv')
else:
    print('No se encontró la carpeta /content/textos. Crea esta estructura para procesar archivos.')


## 6) Visualización rápida

In [ ]:
if 'df_feats' in globals() and len(df_feats):
    for col in ['perplexity','burstiness','ttr','sent_len_avg','word_len_avg','punct_per_100w','functional_pct','flesch']:
        plt.figure()
        df_feats.boxplot(column=col, by='true_label')
        plt.title(f'{col} por clase')
        plt.suptitle('')
        plt.xlabel('Clase real')
        plt.ylabel(col)
        plt.show()


## 7) **Evaluación de detectores** desde CSV

Formato esperado (`results.csv`):

| id_texto | detector | score | etiqueta | true_label |
|---|---|---:|---|---|

- `etiqueta`: salida del detector (`AI`, `HUMAN`, o `MIXTO` si aplica).
- `true_label`: etiqueta real (`ai` o `human`).

In [ ]:
tpl = pd.DataFrame({
    'id_texto': ['ai_01','human_01'],
    'detector': ['DetectorX','DetectorX'],
    'score': [0.87, 0.12],
    'etiqueta': ['AI', 'HUMAN'],
    'true_label': ['ai', 'human']
})
tpl.to_csv('/content/results_template.csv', index=False)
tpl


### Funciones de métricas (TP/FP/FN, Precisión, Recall, F1)

In [ ]:
def compute_confusion(df: pd.DataFrame, positive_label='ai') -> Dict[str, int]:
    y_true = df['true_label'].str.lower()
    y_pred = df['etiqueta'].str.upper().map({'AI':'ai','HUMAN':'human'})
    mask = y_pred.isin(['ai','human'])
    y_true, y_pred = y_true[mask], y_pred[mask]
    TP = int(((y_true==positive_label) & (y_pred==positive_label)).sum())
    FP = int(((y_true!='ai') & (y_pred==positive_label)).sum())
    FN = int(((y_true==positive_label) & (y_pred!='ai')).sum())
    TN = int(((y_true!='ai') & (y_pred!='ai')).sum())
    return {'TP':TP, 'FP':FP, 'FN':FN, 'TN':TN}

def prf_from_counts(c):
    precision = c['TP']/(c['TP']+c['FP']) if (c['TP']+c['FP'])>0 else 0.0
    recall    = c['TP']/(c['TP']+c['FN']) if (c['TP']+c['FN'])>0 else 0.0
    f1        = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0.0
    acc       = (c['TP']+c['TN'])/max(1,(c['TP']+c['TN']+c['FP']+c['FN']))
    return {'precision':precision, 'recall':recall, 'f1':f1, 'accuracy':acc}

def evaluate_by_detector(df_results: pd.DataFrame) -> pd.DataFrame:
    out = []
    for det, sub in df_results.groupby('detector'):
        c = compute_confusion(sub)
        m = prf_from_counts(c)
        out.append({'detector':det, **c, **m})
    return pd.DataFrame(out).sort_values('f1', ascending=False)


### Cargar resultados y evaluar

In [ ]:
RESULTS_PATH = '/content/results.csv'  # reemplaza por tu archivo
try:
    df_res = pd.read_csv(RESULTS_PATH)
    display(df_res.head())
    report = evaluate_by_detector(df_res)
    display(report)
except FileNotFoundError:
    print("Sube tu archivo a /content como 'results.csv' (usa la plantilla generada).")
